In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import logging
from sys import stdout # Esta funcion es peligrosa jajaj permite rev shell
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

[02-06 12:59:14][INFO] answ3r| 2025-02-06 12:59:14.284837


In [5]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)

print(Ab)

[02-06 13:00:57][INFO] 2025-02-06 13:00:57.435572
[02-06 13:00:57][INFO] answ3r| 2025-02-06 13:00:57.435572
[02-06 13:00:57][INFO] 2025-02-06 13:00:57.439444
[02-06 13:00:57][INFO] answ3r| 2025-02-06 13:00:57.439444
[[ 1.  2.  3.  4.  1.]
 [ 2.  5.  6.  7. -2.]
 [ 3.  6.  8.  9.  3.]
 [ 4.  7.  9. 10.  4.]]


In [8]:
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Calcula la inversa de una matriz cuadrada mediante Gauss-Jordan.

    ## Parameters
    `A`: Matriz cuadrada de tamaño n x n.

    ## Return
    `A_inv`: Matriz inversa de A.
    """
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=float)

    assert A.shape[0] == A.shape[1], "La matriz debe ser cuadrada."

    n = A.shape[0]


    Ab = np.hstack((A, np.eye(n)))

    # Aplicar Gauss-Jordan
    for i in range(n):
        p = np.argmax(np.abs(Ab[i:, i])) + i
        if Ab[p, i] == 0:
            raise ValueError("La matriz no es invertible.")

        if p != i:
            logging.debug(f"Intercambiando filas {i} y {p}")
            Ab[[i, p]] = Ab[[p, i]]

        # --- Normalización del pivote
        Ab[i] = Ab[i] / Ab[i, i]

        # --- Eliminación gaussiana
        for j in range(n):
            if i != j:
                Ab[j] -= Ab[j, i] * Ab[i]

    A_inv = Ab[:, n:]  # Extraer la parte derecha que ahora es la inversa
    return A_inv


A = np.array([[2, -3], [-1, 1]])
A_inv = inv_matrix(A)
print(A_inv)

B = np.array([[4, 0, 0, 5], [1, 0, 4, 0], [3, 4, 1, 3],[1, 3, 3, 0]])
B_inv = inv_matrix(B)
print(B_inv)


[[-1. -3.]
 [-1. -2.]]
[[-36.  45.  60. -80.]
 [  3.  -4.  -5.   7.]
 [  9. -11. -15.  20.]
 [ 29. -36. -48.  64.]]
